<a href="https://colab.research.google.com/github/kunalsonalkar/training-llms/blob/main/bytePairEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00


In [2]:
from importlib.metadata import version
import tiktoken
print("tiktoken version: ", version("tiktoken"))

tiktoken version:  0.7.0


In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
tokenizer

<Encoding 'gpt2'>

In [5]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

In [7]:
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [9]:
# Sliding window approach for data masking
with open("/content/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [10]:
# take the first 50 samples
enc_sample = enc_text[50:]

In [11]:
# Decide a context size and then shift input-target pairs
context_size=4
x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [14]:
for i in range(1, context_size+1):
  print(i)
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(context, "---->", desired)

1
[290] ----> 4920
2
[290, 4920] ----> 2241
3
[290, 4920, 2241] ----> 287
4
[290, 4920, 2241, 287] ----> 257


In [25]:
# Printing the actual text
for i in range(1, context_size+1):
  print(i)
  context = enc_sample[:i]
  print(f"The context is:  {context}")
  desired = enc_sample[i]
  print(f"The desired part is: {desired}")
  print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

1
The context is:  [290]
The desired part is: 4920
 and ---->  established
2
The context is:  [290, 4920]
The desired part is: 2241
 and established ---->  himself
3
The context is:  [290, 4920, 2241]
The desired part is: 287
 and established himself ---->  in
4
The context is:  [290, 4920, 2241, 287]
The desired part is: 257
 and established himself in ---->  a


In [31]:
# Efficient data loader for getting inputs and target as PyTorch tensors
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [32]:
# Dataloader to generate batches with input target pairs
def create_dataloader_v1(txt, batch_size=4,
                         max_length=256, stride=128, shuffle=True, drop_last=True):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset, batch_size = batch_size, shuffle=shuffle, drop_last=drop_last)
  return dataloader

In [33]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [34]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [35]:
# batch_size > 1
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print("-------")
print(targets)

tensor([[  379,   407,  1972,   262],
        [   11,   355,  9074,    13],
        [  465,  3996,    13,  2399],
        [  536, 48819,   508, 25722],
        [  290, 45592,    12, 14792],
        [  340,   547,    11,   523],
        [  286, 46431,   465, 27951],
        [ 3150,   339,  8104,   866]])
-------
tensor([[  407,  1972,   262, 18560],
        [  355,  9074,    13,   536],
        [ 3996,    13,  2399,  3656],
        [48819,   508, 25722,   276],
        [45592,    12, 14792,  5986],
        [  547,    11,   523, 16857],
        [46431,   465, 27951,   438],
        [  339,  8104,   866,  1973]])
